In [1]:
import warnings
import itertools
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore")
plt.style.use('fivethirtyeight')
import pandas as pd
import statsmodels.api as sm
import matplotlib
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 20, 14
matplotlib.rcParams['axes.labelsize'] = 14
matplotlib.rcParams['xtick.labelsize'] = 12
matplotlib.rcParams['ytick.labelsize'] = 12
matplotlib.rcParams['text.color'] = 'k'

In [2]:
# Aws 서버에 있는 파일을 가져옴 (,로 값들이 구분된 데이터 파일)
df = pd.read_csv('https://s3.ap-northeast-2.amazonaws.com/data10902/petition/petition.csv', parse_dates=['start','end'])

In [3]:
df.head()

,article_id,start,end,answered,votes,category,title,content
0,21,2017-08-19,2017-11-17,0,9,안전/환경,스텔라 데이지호에 대한 제안입니다.,스텔라 데이지호에 대한 제안입니다.\n3월31일 스텔라 데이지호가 침몰하고 5달째가...
1,22,2017-08-19,2017-11-17,0,17,기타,비리제보처를 만들어주세요.,현 정부에 국민들이 가장 원하는 것은 부패척결입니다. 우리 사회에 각종 비리들이 ...
2,23,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."
3,24,2017-08-19,2017-08-26,0,53,일자리,공공기관 무조건적인 정규직전환을 반대합니다.,현정부에서 정규직 일자리를 늘리는 것에 찬성합니다. 그런데 공공기관 비정규직들은 인...
4,25,2017-08-19,2017-09-03,0,0,미래,제2의 개성공단,"만일 하시는 대통령님 및 각 부처 장관님,주무관님들 안녕하세요!!\n전남 목포에서 ..."


In [4]:
df.shape

(320455, 8)

In [5]:
df.isnull().sum()

article_id    0
start         0
end           0
answered      0
votes         0
category      0
title         0
content       1
dtype: int64

In [6]:
df.describe()

,article_id,answered,votes
count,320455.000000,320455.000000,3.204550e+05
mean,201337.251861,0.000090,1.503033e+02
std,124389.599621,0.009513,4.904435e+03
min,21.000000,0.000000,0.000000e+00
25%,89741.500000,0.000000,1.000000e+00
50%,197075.000000,0.000000,5.000000e+00
75%,311341.500000,0.000000,1.400000e+01
max,417649.000000,1.000000,1.192049e+06


In [7]:
# object 중 가장 많이 나오는 데이터 보여줌
# count와 unique가 일치하지 않는 것은 도배글이 많다는 것을 의미
df.describe(include=np.object)

,category,title,content
count,320455,320455,320454
unique,17,278399,303451
top,정치개혁,이명박 출국금지,이명박 출국금지
freq,50726,3018,597


### 20만건 이상 투표 받은것

In [8]:
df_20 = df.loc[df['votes']>200000]
df_20.shape

(64, 8)

In [9]:
df_20.category.value_counts()

인권/성평등         17
안전/환경           8
기타              7
문화/예술/체육/언론     7
외교/통일/국방        4
정치개혁            4
교통/건축/국토        3
보건복지            3
반려동물            3
경제민주화           2
성장동력            2
육아/교육           2
미래              1
행정              1
Name: category, dtype: int64

In [10]:
df_20[['title','content']].head()

,title,content
1752,청소년이란 이유로 보호법을 악용하는 잔인무도한 청소년들이 늘어나고있습니다. 반드시 ...,안녕하십니까. 청소년보호법이란 명목하에 나쁜짓을 일삼는 청소년들이 너무나 많아지고 ...
10894,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!
18111,낙태죄 폐지와 자연유산 유도약(미프진) 합법화 및 도입을 부탁드립니다.,안녕하세요. 존경하는 대통령님 의원님\n낙태죄 폐지를 청원합니다.\n현재 대한민국은...
25032,'주취감형(술을 먹으면 형벌 감형)' 폐지를 건의(청원)합니다.,"'주취감형'이란, 술을 먹고 범행을 저지를 때, 심신미약 (이성이 없고 우발적)이라..."
43992,"권역외상센터 (이국종 교수님) 추가적, 제도적, 환경적, 인력 지원","청원사항)\n중증외상분야의 추가적, 제도적, 환경적, 인력 지원 방안마련\n현 의료..."


- 20만건 이상의 투표를 받은것 중 상위 5개의 청원

In [11]:
df_20.sort_values(by='votes', ascending=False).head()

,article_id,start,end,answered,votes,category,title,content
313314,408609,2018-10-17,2018-11-16,0,1192049,안전/환경,강서구 피시방 살인 사건. 또 심신미약 피의자입니다.,2018년 10월 14일 엊그제 일어난 강서구 피시방 살인사건에 대한 청원입니다.\...
208597,269548,2018-06-13,2018-07-13,0,714875,외교/통일/국방,"제주도 불법 난민 신청 문제에 따른 난민법, 무사증 입국, 난민신청허가 폐지/개헌 ...",2012년 난민법 제정으로 인해 외국인은 한달 무비자로 입국할 수 있으나 난민신청자...
10894,10949,2017-09-06,2017-12-05,1,615354,미래,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!
118970,142600,2018-02-19,2018-03-21,1,614127,문화/예술/체육/언론,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다","오늘 여자 단체전 팀추월에서 김보름, 박지우 선수는 팀전인데도 불구하고 개인의 영달..."
183791,230552,2018-05-11,2018-06-10,1,419006,인권/성평등,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,최근 홍대 누드크로키 모델의 불법촬영 사건이 있었습니다.\n사건은 굉장히 빠르게 처...


In [12]:
df['answer'] = (df['votes']>200000) == 1 

In [13]:
df.shape

(320455, 9)

In [14]:
df.dtypes

article_id             int64
start         datetime64[ns]
end           datetime64[ns]
answered               int64
votes                  int64
category              object
title                 object
content               object
answer                  bool
dtype: object

In [15]:
df['answer'] = df['answer'].astype('int')

In [16]:
df.dtypes

article_id             int64
start         datetime64[ns]
end           datetime64[ns]
answered               int64
votes                  int64
category              object
title                 object
content               object
answer                 int32
dtype: object

- 답변대상 청원중 아직 답변되지 않은 청원의 수를 계산

In [17]:
df['answer_diff'] = df['answer'] - df['answered']
df['answer_diff'].sum()

35

In [18]:
answered_df = df.loc[df['answer'] == 1]
answered_df.sort_values('votes', ascending=False).head(5)

,article_id,start,end,answered,votes,category,title,content,answer,answer_diff
313314,408609,2018-10-17,2018-11-16,0,1192049,안전/환경,강서구 피시방 살인 사건. 또 심신미약 피의자입니다.,2018년 10월 14일 엊그제 일어난 강서구 피시방 살인사건에 대한 청원입니다.\...,1,1
208597,269548,2018-06-13,2018-07-13,0,714875,외교/통일/국방,"제주도 불법 난민 신청 문제에 따른 난민법, 무사증 입국, 난민신청허가 폐지/개헌 ...",2012년 난민법 제정으로 인해 외국인은 한달 무비자로 입국할 수 있으나 난민신청자...,1,1
10894,10949,2017-09-06,2017-12-05,1,615354,미래,조두순 출소반대,제발 조두순 재심다시해서 무기징역으로 해야됩니다!!!,1,0
118970,142600,2018-02-19,2018-03-21,1,614127,문화/예술/체육/언론,"김보름, 박지우 선수의 자격박탈과 적폐 빙상연맹의 엄중 처벌을 청원합니다","오늘 여자 단체전 팀추월에서 김보름, 박지우 선수는 팀전인데도 불구하고 개인의 영달...",1,0
183791,230552,2018-05-11,2018-06-10,1,419006,인권/성평등,여성도 대한민국 국민입니다. 성별 관계없는 국가의 보호를 요청합니다.,최근 홍대 누드크로키 모델의 불법촬영 사건이 있었습니다.\n사건은 굉장히 빠르게 처...,1,0


### 어느 분야의 청원이 가장 많이 올라왔나

In [19]:
category = pd.DataFrame(df['category'].value_counts()).reset_index()

In [20]:
category.columns = ['category','counts']

In [21]:
category

,category,counts
0,정치개혁,50726
1,기타,39284
2,인권/성평등,29071
3,안전/환경,25269
4,교통/건축/국토,22445
5,외교/통일/국방,21712
6,육아/교육,21333
7,보건복지,19885
8,일자리,18012
9,행정,16553


### 청원이 가장 많이 들어 온 날은 언제인지 정렬

In [22]:
start_df = pd.DataFrame(df['start'].value_counts()).reset_index()
start_df.columns = ['start', 'count']
start_df = start_df.sort_values('count', ascending=False)
print('청원 집계: {}일'.format(start_df.shape[0]))
start_df.head()

청원 집계: 431일


,start,count
0,2017-11-11,9623
1,2017-09-05,5952
2,2018-01-11,3368
3,2018-02-06,2631
4,2017-11-09,2487


### 피봇 테이블로 투표를 가장 많이 받은 분야 찾기

In [23]:
petitions_unique = pd.pivot_table(df, index=['category'], aggfunc=np.sum)

In [24]:
petitions_best = petitions_unique.sort_values(by='votes', \
                                              ascending=False).reset_index()
petitions_best

,category,answer,answer_diff,answered,article_id,votes
0,인권/성평등,17,8,9,5211207315,9271075
1,안전/환경,8,6,2,5041573662,5398863
2,정치개혁,4,2,2,8987480171,4510687
3,기타,7,6,1,7841416124,4334342
4,문화/예술/체육/언론,7,3,4,3106421706,3897089
5,보건복지,3,1,2,4270378553,3794485
6,육아/교육,2,1,1,3891551302,3699072
7,외교/통일/국방,4,3,1,4603751751,3392805
8,교통/건축/국토,3,1,2,5887815225,2284821
9,반려동물,3,3,0,708435716,1678026


## 투표를 가장 많이 받은 날

In [25]:
petitions_start = pd.pivot_table(df, index=['start'], aggfunc=np.sum)
votes_df = petitions_start.sort_values(by='votes', ascending=False)
votes_df.loc[petitions_start['votes'] > 350000]

,answer,answer_diff,answered,article_id,votes
start,,,,,
2018-10-17,1,1,0,378900198,1300523
2018-06-13,1,1,0,146187973,786157
2018-10-18,2,2,0,423557789,721524
2018-02-19,1,0,1,99332898,701520
2018-07-22,1,1,0,282639463,672491
2017-09-06,1,0,1,22268570,648209
2018-06-24,2,2,0,359345089,628925
2018-10-04,2,1,1,290746555,626761
2018-02-23,2,0,2,168561151,608530


## 청원이 많이 올라온날 vs 투표를 많이 한날

In [26]:
# 두 개의 데이터프레임을 합치기 위해 인덱스를 생성한다.
votes_df = votes_df.reset_index()
hottest_day_df = start_df.merge(votes_df, on='start', how='left')
hottest_day_df.sort_values('count', ascending=False)[:5]

,start,count,answer,answer_diff,answered,article_id,votes
0,2017-11-11,9623,0,0,0,348559310,85074
1,2017-09-05,5952,0,0,0,38414241,48808
2,2018-01-11,3368,0,0,0,291069195,44570
3,2018-02-06,2631,0,0,0,342371897,83038
4,2017-11-09,2487,0,0,0,73459579,34774


In [27]:
hottest_day_df.sort_values('votes', ascending=False)[:5]

,start,count,answer,answer_diff,answered,article_id,votes
100,2018-10-17,926,1,1,0,378900198,1300523
299,2018-06-13,542,1,1,0,146187973,786157
66,2018-10-18,1032,2,2,0,423557789,721524
211,2018-02-19,698,1,0,1,99332898,701520
106,2018-07-22,898,1,1,0,282639463,672491


## 시계열데이터

In [28]:
df['start_month'] = df['start'].dt.month
df['start_day'] = df['start'].dt.day
df['start_hour'] = df['start'].dt.hour
df['start_dow'] = df['start'].dt.dayofweek
df.shape

(320455, 14)

- 청원이 가장 많이 들어온 달
- 요일별 청원 수

In [29]:
crypto = df[( df.title.str.find('가상화폐') != -1 ) | ( df.content.str.find('가상화폐') != -1  )]
crypto.shape

(7863, 14)

In [30]:
crypto.head()

,article_id,start,end,answered,votes,category,title,content,answer,answer_diff,start_month,start_day,start_hour,start_dow
12781,12841,2017-09-08,2017-12-07,0,3,성장동력,가상전자화폐를 인정하고 규제해야합니다,가상전자화폐\n흔히 코인이라 불리는 비생산적이고 소모적이며\n시장을 혼란케 하고 산...,0,0,9,8,0,4
19325,19575,2017-10-06,2017-11-05,0,5,성장동력,ICO 전면금지에 대한 입장,정부는 9월 29일 김용범 금융위원회 부위원장 주재로 <가상통화 관계기관 합동TF>...,0,0,10,6,0,4
20874,21277,2017-10-15,2017-11-14,0,3906,미래,대통령님에게 전하는 지부상소(持斧上疏)입니다. -블록체인 기술에 대한 이야기-,지부상소(持斧上疏)를 이야기할 만큼 간절하게 원합니다.\n이 글이 문재인 대통령님께...,0,0,10,15,0,6
20911,21327,2017-10-15,2017-11-14,0,3,경제민주화,가상화폐 투기 광풍을 막아주세요,가상화폐 규제 어떻게 해야 하나 ?\n이제 우리 앞에 놓여진 사회적 이슈가 되어\n...,0,0,10,15,0,6
26541,27765,2017-11-08,2017-12-08,0,1,경제민주화,가상화폐 투자대행회사의 무분별한 수수료 바로잡아주세요,가상화폐 ico로 대리업무를 대행해주고 5-30% 수수료를 책정하고 계약서나 기타 ...,0,0,11,8,0,2


In [31]:
crypto['category'].value_counts()

경제민주화          1964
미래             1890
성장동력           1408
기타             1006
정치개혁            874
행정              344
일자리             113
인권/성평등           65
안전/환경            54
외교/통일/국방         43
교통/건축/국토         38
문화/예술/체육/언론      27
보건복지             14
저출산/고령화대책         8
육아/교육             8
반려동물              5
농산어촌              2
Name: category, dtype: int64

In [32]:
import re
p = r'.*(돌봄|아이|초등|보육).*'
care = df[df['title'].str.match(p) |
           df['content'].str.match(p, flags=re.MULTILINE)]
care.shape

(38956, 14)